Creating testing and training sets of data

In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import warnings; warnings.simplefilter('ignore')

In [61]:
movies_df = pd.read_csv("TheMoviesDataset\movies_metadata.csv")
ratings_df = pd.read_csv("TheMoviesDataset/ratings_small.csv")
links_df = pd.read_csv("TheMoviesDataset\links_small.csv")

In [62]:
movies_df.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [63]:
ratings_df.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [64]:
links_df.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [65]:
movies_df = movies_df.rename(columns={'id': 'movieId'})

In [66]:
#incorrectly formated data found within dataset, easier to drop movies by index than fix

movies_df = movies_df.drop(35586)
movies_df = movies_df.drop(35587)
movies_df = movies_df.drop(29502)
movies_df = movies_df.drop(29503)
movies_df = movies_df.drop(19729)
movies_df = movies_df.drop(19730)

#reset index so that theres none missing
movies_df.reset_index(drop=True, inplace=True)

In [67]:
movies_df['movieId'] = movies_df['movieId'].astype('int')
ratings_df['movieId'] = ratings_df['movieId'].astype('int')
links_df = links_df[links_df['tmdbId'].notnull()]['tmdbId'].astype('int')


In [68]:
#reducing dataset to only ranked movies

movies_df = movies_df[movies_df['movieId'].isin(links_df)]
ratings_df = ratings_df[ratings_df['movieId'].isin(links_df)]

print('movies: ', movies_df.shape)
print('ratings: ', ratings_df.shape)
print('links_small: ', links_df.shape)

movies:  (9099, 24)
ratings:  (32146, 4)
links_small:  (9112,)


In [69]:
ratings_df = pd.merge(ratings_df, movies_df[['movieId', 'title']], on='movieId' , how='left')
ratings_df = ratings_df.drop('timestamp', axis=1)
ratings_df = ratings_df[['userId','movieId','title', 'rating']]
ratings_df.head(5)

,userId,movieId,title,rating
0,1,1371,Rocky III,2.5
1,1,1405,Greed,1.0
2,1,2105,American Pie,4.0
3,1,2193,My Tutor,2.0
4,1,2294,Jay and Silent Bob Strike Back,2.0


In [77]:
train_data, test_data = train_test_split(ratings_df, test_size=0.2)

In [78]:
train_data = train_data.sort_values(['userId', 'movieId'])
train_data.head()

,userId,movieId,title,rating
0,1,1371,Rocky III,2.5
2,1,2105,American Pie,4.0
3,1,2193,My Tutor,2.0
4,1,2294,Jay and Silent Bob Strike Back,2.0
5,2,62,2001: A Space Odyssey,3.0


In [79]:
test_data = test_data.sort_values(['userId','movieId'])
test_data.head()

,userId,movieId,title,rating
1,1,1405,Greed,1.0
23,2,273,The White Sound,4.0
30,2,350,The Devil Wears Prada,4.0
34,2,454,Romeo + Juliet,4.0
35,2,468,My Own Private Idaho,4.0


In [80]:
train_data.to_csv('training_data.csv', index = False)
test_data.to_csv('testing_data.csv', index = False)